# Assignment 2: Entity Resolution (Part 1)

## Objective

Data scientists often spend 80% of their time on [data preparation](https://www.infoworld.com/article/3228245/the-80-20-data-science-dilemma.html). If your career goal is to become a data scientist, you have to master data cleaning and data integration skills. In this assignment, you will learn how to solve the Entity Resolution (ER) problem, a very common problem in data cleaning and integration. After completing this assignment, you should be able to answer the following questions:

1. What is ER?
2. What are the applications of ER in data integration and cleaning? 
3. How to avoid $n^2$ comparisons? 
4. How to compute Jaccard Similarity?
5. How to evaluate an ER result?

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

The data for Assignment 2 (Part 1 and Part 2) can be downloaded from [A2-data.zip](A2-data.zip).

## Overview

ER is defined as finding different records that refer to the same real-world entity, e.g., iPhone 4-th generation vs. iPhone four. It is central to data integration and cleaning. In this assignment, you will learn how to apply ER in a data integration setting. But the program that you are going to write can be easily extended to a data-cleaning setting, being used to detect _duplication values_.   

Imagine that you want to help your company's customers to buy products at a cheaper price. In order to do so, you first write a [web scraper](https://nbviewer.jupyter.org/github/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1-1.ipynb) to crawl product data from Amazon.com and Google Shopping, respectively, and then integrate the data together. Since the same product may have different representations in the two websites, you are facing an ER problem. 

Existing ER techniques can be broadly divided into two categories: similarity-based (Part 1) and learning-based (Part 2). 


## Similarity Join

Unlike a learning-based technique, a similarity-based technique (a.k.a similarity join) does not need any label data. It first chooses a similarity function and a threshold, and then returns the record pairs whose similarity values are above the threshold. These returned record pairs are thought of as matching pairs, i.e., referring to the same real-world entity. 

Depending on particular applications, you may need to choose different similarity functions. In this assignment, we will use Jaccard similarity, i.e., $\textsf{Jaccard}(r, s) = \big|\frac{r \cap s}{r \cup s}\big|$. Here is the formal definition of this problem.

> **Jaccard-Similarity Join**: Given two DataFrames, R and S, and a threshold $\theta \in (0, 1]$, the jaccard-similarity join problem aims to find all record pairs $(r, s) \in R \times S$ such that $\textsf{Jaccard}(r, s) \geq \theta$  

To implement similarity join, you need to address the following challenges:

1. Jaccard is used to quantify the similarity between two sets instead of two records. You need to convert each record to a set.

2. A naive implementation of similarity join is to compute Jaccard for all $|R \times S|$ possible pairs. Imagine R and S have one million records. This requires doing 10^12 pair comparisons, which is extremely expensive. Thus, you need to know how to avoid n^2 comparisons. 

3. The output of ER is a set of matching pairs, where each pair is considered as referring to the same real-world entity. You need to know how to evaluate the quality of an ER result.

Next, you will be guided to complete four tasks. After finishing these tasks, I suggest you going over the above challenges again, and understand how they are addressed.

Read the code first, and then implement the remaining four functions: `preprocess_df`, `filtering`, `verification`, and `evaluate` by doing Tasks A-D, respectively.

``` python
# similarity_join.py
import re
import pandas as pd

class SimilarityJoin:
    def __init__(self, data_file1, data_file2):
        self.df1 = pd.read_csv(data_file1)
        self.df2 = pd.read_csv(data_file2)
          
    def preprocess_df(self, df, cols): 
        """
            Write your code!
        """ 
    
    def filtering(self, df1, df2):
        """
            Write your code!
        """
      
    def verification(self, cand_df, threshold):
        """
            Write your code!
        """
        
    def evaluate(self, result, ground_truth):
        """
            Write your code!
        """
        
    def jaccard_join(self, cols1, cols2, threshold):
        new_df1 = self.preprocess_df(self.df1, cols1)
        new_df2 = self.preprocess_df(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.shape[0] *self.df2.shape[0])) 
        
        cand_df = self.filtering(new_df1, new_df2)
        print ("After Filtering: %d pairs left" %(cand_df.shape[0]))
        
        result_df = self.verification(cand_df, threshold)
        print ("After Verification: %d similar pairs" %(result_df.shape[0]))
        
        return result_df
       
        

if __name__ == "__main__":
    er = SimilarityJoin("Amazon_sample.csv", "Google_sample.csv")
    amazon_cols = ["title", "manufacturer"]
    google_cols = ["name", "manufacturer"]
    result_df = er.jaccard_join(amazon_cols, google_cols, 0.5)

    result = result_df[['id1', 'id2']].values.tolist()
    ground_truth = pd.read_csv("Amazon_Google_perfectMapping_sample.csv").values.tolist()
    print ("(precision, recall, fmeasure) = ", er.evaluate(result, ground_truth))
```


The program will output the following when running on the sample data:


> Before filtering: 256 pairs in total

> After Filtering: 84 pairs left

> After Verification: 6 similar pairs

> (precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


### Task A. Data Preprocessing (Record --> Token Set)

Since Jaccard needs to take two sets as input, your first job is to preprocess DataFrames by transforming each record into a set of tokens. Please implement the following function.   

```python
def preprocess_df(self, df, cols): 
    """ 
        Input: $df represents a DataFrame
               $cols represents the list of columns (in $df) that will be concatenated and be tokenized

        Output: Return a new DataFrame that adds the "joinKey" column to the input $df

        Comments: The "joinKey" column is a list of tokens, which is generated as follows:
                 (1) concatenate the $cols in $df; 
                 (2) apply the tokenizer to the concatenated string
        Here is how the tokenizer should work:
                 (1) Use "re.split(r'\W+', string)" to split a string into a set of tokens
                 (2) Convert each token to its lower-case
    """ 
```

In [2]:
import pandas as pd
import re

In [25]:

def preprocess_df(df, cols): 
    new_df = df.copy()
    joinString = ''
    for col in cols:
        joinString += new_df[col].apply(lambda x: x if pd.notna(x) else '') + ' '

    # joinKey = re.split(r'\W+', joinString)
    # joinString = [joinKey.lower() for key in joinKey if key]
    new_df['joinKey'] = joinString.apply(lambda x: re.split(r'\W+', x.lower())).apply(lambda lst: [i for i in lst if i])
    return new_df



amazon_cols = ["title", "manufacturer"]
google_cols = ["name", "manufacturer"]

# preprocess_df(df1, amazon_cols).to_csv('test1.csv')
# preprocess_df(df2, google_cols).to_csv('test2.csv')




For the purpose of testing, you can compare your outputs with new_df1 and new_df2 that can be found from the `Amazon-Google-Sample` folder.

### Task B. Filtering Obviously Non-matching Pairs

To avoid $n^2$ pair comparisons, ER algorithms often follow a filtering-and-verification framework. The basic idea is to first filter obviously non-matching pairs and then only verify the remaining pairs.  

In Task B, your job is to implement the <font color="blue">filtering</font> function. This function will filter all the record pairs whose joinKeys do not share any token. This is because based on the definition of Jaccard, we can deduce that **if two sets do not share any element (i.e., $r\cap s = \phi$), their Jaccard similarity values must be zero**. Thus, we can safely remove them. 

```python
def filtering(self, df1, df2):
    """ 
        Input: $df1 and $df2 are two input DataFrames, where each of them 
               has a 'joinKey' column added by the preprocess_df function

        Output: Return a new DataFrame $cand_df with four columns: 'id1', 'joinKey1', 'id2', 'joinKey2',
                where 'id1' and 'joinKey1' are from $df1, and 'id2' and 'joinKey2'are from $df2.
                Intuitively, $cand_df is the joined result between $df1 and $df2 on the condition that 
                their joinKeys share at least one token. 

        Comments: Since the goal of the "filtering" function is to avoid n^2 pair comparisons, 
                  you are NOT allowed to compute a cartesian join between $df1 and $df2 in the function. 
                  Please come up with a more efficient algorithm (see hints in Lecture 2). 
    """
```

In [11]:
df2.head()

,id,name,description,manufacturer,price,joinKey
0,http://www.google.com/base/feeds/snippets/1102...,punch software 20100 punch! 5 in 1 home design,total interior and exterior home design with r...,punch software,35.99,"[punch, software, 20100, punch, 5, in, 1, home..."
1,http://www.google.com/base/feeds/snippets/7249...,onone software ice-10770 intellihance pro 4.1 ...,dynamic image enhancement and color correction...,onone software,158.99,"[onone, software, ice, 10770, intellihance, pr..."
2,http://www.google.com/base/feeds/snippets/1227...,adobe cs3 web standard,system requirements powerpc g4 or g5 or intel ...,NaN,1035.99,"[adobe, cs3, web, standard]"
3,http://www.google.com/base/feeds/snippets/1835...,red mile entertainment 124 - aircraft power pa...,red mile entertainment 124 : with firepower fo...,NaN,27.97,"[red, mile, entertainment, 124, aircraft, powe..."
4,http://www.google.com/base/feeds/snippets/3409...,sonicwall gms 1000n-incremental lic upg,today enterprises and service providers face i...,NaN,62920.89,"[sonicwall, gms, 1000n, incremental, lic, upg]"


In [36]:
df1 = pd.read_csv("Amazon_sample.csv")
df2 = pd.read_csv("Google_sample.csv")
df1 = preprocess_df(df1, amazon_cols)
df2 = preprocess_df(df2, google_cols)

df1 = df1[['id', 'joinKey']].rename(columns={'id': 'id_1'})
df2 = df2[['id', 'joinKey']].rename(columns={'id': 'id_2'})
df1_n = df1.explode('joinKey')
df2_n = df2.explode('joinKey')

df_m = df1_n.merge(df2_n, on='joinKey')
df_keep_ids = df_m.drop_duplicates(subset=['id_1', 'id_2'])[['id_1', 'id_2']]

cand_df = df_keep_ids.merge(df1, on='id_1').rename(columns={'joinKey': 'joinKey_1'})
cand_df = cand_df.merge(df2, on='id_2').rename(columns={'joinKey': 'joinKey_2'})
cand_df = cand_df[['id_1', 'joinKey_1', 'id_2', 'joinKey_2']]
print(cand_df)

'''
#sub_df on id_1, id_2
df_selected = df_merged
#




#join

#

cand_df = pd.merge(df1, df_selected, on='id_1')

cand_df = pd.merge(df2, cand_df, on='id_2')

cand_df = cand_df[['id_1', 'joinKey_1', 'id_2', 'joinKey_2']]
'''

          id_1                                          joinKey_1  \
0   b0002mg5uk         [iview, mediapro, 2, 5, global, marketing]   
1   b000v7vz1u  [onone, essentials, for, adobe, photoshop, ele...   
2   b0002mg5uk         [iview, mediapro, 2, 5, global, marketing]   
3   b000v7vz1u  [onone, essentials, for, adobe, photoshop, ele...   
4   b00002s6sc   [punch, 5, in, 1, home, design, punch, software]   
..         ...                                                ...   
79  b000ndibge  [adobe, creative, suite, cs3, web, standard, m...   
80  b000in8n30                       [hijack2, navarre, software]   
81  b00013wh0w  [handmark, oxford, american, desk, dictionary,...   
82  b000gaqlxe  [aquazone, bass, edition, smith, micro, software]   
83  b00006hvvo  [upg, sgms, 1000, incremental, node, sonic, sy...   

                                                 id_2  \
0   http://www.google.com/base/feeds/snippets/1267...   
1   http://www.google.com/base/feeds/snippets/1267...   


"\n#sub_df on id_1, id_2\ndf_selected = df_merged\n#\n\n\n\n\n#join\n\n#\n\ncand_df = pd.merge(df1, df_selected, on='id_1')\n\ncand_df = pd.merge(df2, cand_df, on='id_2')\n\ncand_df = cand_df[['id_1', 'joinKey_1', 'id_2', 'joinKey_2']]\n"

In [33]:
df1.head()

,id_1,joinKey
0,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]"
1,b0002jtvng,"[bias, deck, le, 3, 5, macintosh, cd, bias]"
2,b0007lw22g,"[apple, ilife, 06, mac, dvd, older, version, a..."
3,b00007kh02,"[extensis, intellihance, pro, 4, x, win, mac, ..."
4,b000saufpw,"[dk, amazing, animals, 1, 1, global, software,..."


In [54]:
def filtering(df1, df2):
    id1 = df1['id']
    joinKey1 = df1['joinKey']
    len1 = len(id1)
    id2 = df2['id']
    joinKey2 = df2['joinKey']
    len2 = len(id2)

    column_names = ['id1', 'joinKey1', 'id2', 'joinKey2']
    df = pd.DataFrame(columns = column_names)

    for i in range(len1):
        for j in range(len2):
            if set(joinKey1[i]).intersection(set(joinKey2[j])):
                df = df.append({'id1':id1[i], 'joinKey1':joinKey1[i], 'id2':id2[j], 'joinKey2':joinKey2[j]}, ignore_index=True)

    return df

tdf1 = preprocess_df(df1, amazon_cols)
tdf2 = preprocess_df(df2, google_cols)
filtering(tdf1, tdf2).to_csv('cand_df.csv', index=False)

For the purpose of testing, you can compare your output with cand_df that can be found from the `Amazon-Google-Sample` folder.

### Task C. Computing Jaccard Similarity for Survived Pairs

In the second phase of the filtering-and-verification framework, we will compute the Jaccard similarity for each survived pair and return those pairs whose jaccard similarity values are no smaller than the specified threshold.

In Task C, your job is to implement the <font color="blue">verification</font> function. This task looks simple, but there are a few small "traps". 


```python
def verification(self, cand_df, threshold):
        """ 
            Input: $cand_df is the output DataFrame from the 'filtering' function. 
                   $threshold is a float value between (0, 1] 

            Output: Return a new DataFrame $result_df that represents the ER result. 
                    It has five columns: id1, joinKey1, id2, joinKey2, jaccard 

            Comments: There are two differences between $cand_df and $result_df
                      (1) $result_df adds a new column, called jaccard, which stores the jaccard similarity 
                          between $joinKey1 and $joinKey2
                      (2) $result_df removes the rows whose jaccard similarity is smaller than $threshold 
        """
```

In [73]:
cand_df = filtering(tdf1, tdf2)
cand_df.loc[0:1]

,id1,joinKey1,id2,joinKey2
0,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]",http://www.google.com/base/feeds/snippets/1102...,"[punch, software, 20100, punch, 5, in, 1, home..."
1,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]",http://www.google.com/base/feeds/snippets/5572...,"[bias, deck, 3, 5, le, digital, audio, recordi..."


In [78]:
def verification(cand_df, threshold):
    new_df = cand_df.copy()
    new_df['jaccard'] = new_df.apply(lambda x: len(set(x['joinKey1']) & set(x['joinKey2']))/len(set(x['joinKey1']) | set(x['joinKey2'])), axis=1)
    return new_df[new_df.jaccard>=threshold]

result_df = verification(cand_df, 0.5)
result_df.to_csv('result_df.csv', index=False)

For the purpose of testing, you can compare your output with result_df that can be found from the `Amazon-Google-Sample` folder.

### Task D. Evaluating an ER result

How should we evaluate an ER result? Before answering this question, let's first recall what the ER result looks like. The goal of ER is to identify all matching record pairs. Thus, the ER result should be a set of identified matching pairs, denoted by R. One thing that we want to know is that what percentage of the pairs in $R$ that are truly matching? This is what Precision can tell us. Let $T$ denote the truly matching pairs in $R$. Precision is defined as:
$$Precision = \frac{|T|}{|R|}$$

In addition to Precision, another thing that we care about is that what percentage of truly matching pairs that are identified. This is what Recall can tell us. Let $A$ denote the truly matching pairs in the entire dataset. Recall is defined as: 

$$Recall = \frac{|T|}{|A|}$$

There is an interesting trade-off between Precision and Recall. As more and more pairs that are identified as matching, Recall increases while Precision potentially decreases. For the extreme case, if we return all the pairs as matching pairs, we will get a perfect Recall (i.e., Recall = 100%) but precision will be the worst. Thus, to balance Precision and Recall, people often use FMeasure to evaluate an ER result:

$$FMeasure = \frac{2*Precision*Recall}{Precision+Recall}$$

In Task D, you will be given an ER result as well as the ground truth that tells you what pairs are truly matching. Your job is to calculate Precision, Recall and FMeasure for the result. 
```python
def evaluate(self, result, ground_truth):
    """ 
        Input: $result is a list of matching pairs identified by the ER algorithm
               $ground_truth is a list of matching pairs labeld by humans

        Output: Compute precision, recall, and fmeasure of $result based on $ground_truth, and
                return the evaluation result as a triple: (precision, recall, fmeasure)

    """
```


In [100]:
r_lst = result_df[['id1', 'id2']].values.tolist()
r_lst = [x[0]+'-'+x[1] for x in r_lst]
# r_lst

a_lst = pd.read_csv("Amazon_Google_perfectMapping_sample.csv").values.tolist()
a_lst = [x[0]+'-'+x[1] for x in a_lst]
# a_lst

t_lst = set(r_lst) & set(a_lst)
# t_lst

In [37]:
result = [(1, 'apple'), (2, 'banana'), (3, 'orange')]
result_first_elements = list(map(lambda x: x[0], result))
result_first_elements

[1, 2, 3]

(precision, recall, fmeasure) = (1.0, 0.375, 0.5454545454545454)

In [101]:
precision = len(t_lst) / len(r_lst)

recall = len(t_lst) / len(a_lst)

fmeasure = 2 * precision * recall / (precision + recall)

print(precision, recall, fmeasure)

1.0 0.375 0.5454545454545454


## Submission

Implement `preprocess_df`, `filtering`, `verification`, and `evaluate` functions in `similarity_join.py`. Submit your code file (`similarity_join.py`)  to the CourSys activity Assignment 2.